In [ ]:
import sys
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.python.client import device_lib
import os

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('DEVICES', device_lib.list_local_devices())

# Configuration
BATCH_SIZE = 32
EPOCHS = 5

# Load the Fashion MNIST dataset
datasets, info = tfds.load('fashion_mnist', with_info=True, as_supervised=True)

# Normalize and batch process the dataset
ds_train = datasets['train'].map(
    lambda x, y: (tf.cast(x, tf.float32)/255.0, y)
).batch(BATCH_SIZE)

ds_test = datasets['test'].map(
    lambda x, y: (tf.cast(x, tf.float32)/255.0, y)
).batch(BATCH_SIZE)

# Build the CNN model (same architecture as in the notebook)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Display model summary
model.summary()

# Train the model
print("Starting training...")
history = model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_test,
    verbose=1
)

In [ ]:
# Evaluate the model
print("Evaluating model...")
test_loss, test_accuracy = model.evaluate(ds_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")

# Save the model locally
MODEL_DIR = "./saved_model.keras"
model.save(MODEL_DIR)
print(f"Model saved to {MODEL_DIR}")

# Optional: Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['sparse_categorical_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()